In [1]:
# get api key from your .env file

import os

from dotenv import load_dotenv

from pathlib import Path

 

dotenv_path = Path('./SpotifyAPIKey.env')

load_dotenv(dotenv_path=dotenv_path)


#load_dotenv()

API_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
API_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
 
print(API_CLIENT_ID)
print(API_CLIENT_SECRET)

3f7295d24d8041688f494196baeb607f
42896ec37b22446680c5a485858a4843


In [2]:
import base64
import requests
import pandas as pd

In [3]:
#Client Credential Workflow Setup

client_creds = f'{API_CLIENT_ID}:{API_CLIENT_SECRET}'
type(client_creds)

client_creds_b64 = base64.b64encode(client_creds.encode())

In [4]:
#More Workflow setup
token_url = 'https://accounts.spotify.com/api/token'
method = 'POST'
token_data = {
    'grant_type' : 'client_credentials'
}
token_headers = {
    'Authorization':f'Basic {client_creds_b64.decode()}',
    'Content-Type':'application/x-www-form-urlencoded'
}

In [5]:
#Authentication request to get bearer token
r = requests.post(token_url, data=token_data, headers=token_headers)
response = r.json()

access_token = response['access_token']

In [6]:
#header for each request using the token as authorization

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [7]:
#Testing get request for Search on Bohemian Rhapsody by Queen.

base_url = 'https://api.spotify.com/v1/'
r_test = requests.get(base_url+"search?q=track: amanda&type=track", headers=headers)
r_test = r_test.json()
print(r_test)

#this could be an issue.. there could be many songs that come up with just the artist / track data..

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3A+amanda&type=track&offset=0&limit=20', 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/29kkCKKGXheHuoO829FxWK'}, 'href': 'https://api.spotify.com/v1/artists/29kkCKKGXheHuoO829FxWK', 'id': '29kkCKKGXheHuoO829FxWK', 'name': 'Boston', 'type': 'artist', 'uri': 'spotify:artist:29kkCKKGXheHuoO829FxWK'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ'

I need to get the most popular track with this name. 
It would stand to reason that is the track that belong to the Billboard hit. 
Through inspection, I see the track URI is near the bottom 'type': 'track', 
'uri': 'spotify:track:7tFiyTwD0nx5a1eklYtX2J'}
But how do I extract this....?

Through further inspection I see that when multiple tracks are searched, 
they are sorted by popularity. 

In [21]:
#Time to get the URIs
#Using a Json formatter online, I see that there are lists nested in the main dictionary. 
#I will decompose the dictionary and then use a compresed for-loop to get the URI which is the ID key:value.



hazzah!

In [22]:
#Now let's make a function that I can use in a for loop to get a df of all the URIs for the hit songs. 
#I am scared that this might miss lots of songs.. I will need to find a way to isolate songs that are not picked up. 
#ugh

def GetSongURI(titleName,artistName):
    song_data = requests.get(base_url+f"search?q=track:{titleName}%20{artistName}&type=track", headers=headers)
    song_data = song_data.json()
    result = song_data['tracks']['items']
    
    for items in result:
        for artist in items['artists']:
            if artist['name'].lower() == artistName.lower():
                uri = items['id']
                return uri


    track = requests.get(base_url+f"search?q=track:{titleName}&type=track", headers=headers)
    track = track.json()
    result = track['tracks']['items']

    for items in result:
        for artist in items['artists']:
            if artist['name'].lower() == artistName.lower():
                uri = items['id']
                return uri

    for items in result:
        for artist in items['artists']:
            if artist['name'].lower() in artistName.lower():
                uri = items['id']
                return uri


In [23]:
test_2 = GetSongURI('Umbrella', 'Rihanna')

In [24]:
print(test_2)

49FYlytm3dAAraYgpoJZux


In [9]:
#Confirm test_2 worked

umbrella = requests.get('https://api.spotify.com/v1/tracks/74atKkOasLOVzvqB6mYgga', headers=headers)
umbrella = umbrella.json()
print(umbrella)

{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'}, 'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4', 'id': '3TVXtAsR1Inumwj472S9r4', 'name': 'Drake', 'type': 'artist', 'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'ME', 'MG', 'MH', 'MK', 'ML', 'MN

In [162]:
#Let's loop through the Billboards Hit dataset and add the URIs
#Actually first lets test it for the first row. Hanging By A Moment, Lifehouse

df = pd.read_csv('BillboardFullDataset.csv',header=0, index_col = 0)

df = df[['Title','Artist']]

df['Title'] = df['Title'].str.lower()
df['Artist'] = df['Artist'].str.lower()

df.head(7)



,Title,Artist
0,hanging by a moment,lifehouse
1,fallin’,alicia keys
2,all for you,janet jackson
3,drops of jupiter (tell me),train
4,i’m real,jennifer lopez feat. ja rule
5,if you’re gone,matchbox twenty
6,let me blow ya mind,eve feat. gwen stefani


In [165]:
def append_URIs(dataframe):
    for lab, row in dataframe.iterrows():
        dataframe.loc[lab, 'URI'] = GetSongURI(row['Title'],row['Artist'])

In [166]:
append_URIs(df)

KeyError: 'tracks'

In [ ]:
df.describe()

In [494]:
missing = df.loc[df['URI']isna()]

We have an issue.. After checking if this lamda function works my worst fears have come true. My GetSongURI function will error out if the search query doesn't pick anything up. I will need to find a way to search for both songs and artist separately to make sure that all track URIs are picked up. 

Moving to a new workbook. This is a shit show. 